# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [5]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [6]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [7]:
year: int = 2020
topic: str = "MEDICINA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [8]:
add_list: list = [x for x in range(26, 30, 1)] + [31, 32, 33, 35, 37, 78, 81, 169]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [9]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. Pregunta asociada a la imagen 1.
3,
4,Paciente miope que consulta por pérdida de vis...
5,ojo derecho. La exploración del fondo de ojo s...
6,serva en la imagen. ¿Cuál de las siguientes af...
7,ciones es correcta respecto a este paciente?:
8,
9,1. Tiene una mejoría de los síntomas al desper...


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [10]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,1,38,1,75,3,112,3.0,149,4.0
1,2,3,39,2,76,3,113,3.0,150,1.0
2,3,2,40,3,77,1,114,4.0,151,2.0
3,4,4,41,3,78,3,115,1.0,152,1.0
4,5,3,42,2,79,1,116,4.0,153,1.0
5,6,3,43,4,80,2,117,3.0,154,1.0
6,7,4,44,3,81,1,118,3.0,155,2.0
7,8,4,45,2,82,4,119,1.0,156,2.0
8,9,2,46,1,83,1,120,3.0,157,3.0
9,10,3,47,4,84,3,121,4.0,158,1.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [11]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,35,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [12]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [13]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [14]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [15]:
raw_exam_df.head(10)

0                  1. Pregunta asociada a la imagen 1.
1    Paciente miope que consulta por pérdida de vis...
2    ojo derecho. La exploración del fondo de ojo s...
3    serva en la imagen. ¿Cuál de las siguientes af...
4        ciones es correcta respecto a este paciente?:
5    1. Tiene una mejoría de los síntomas al desper...
6                    que empeoran según avanza el día.
7    2. Refiere una pérdida de visión, como una cor...
8    que le afecta a la zona superior del campo vis...
9    3. Sería necesario realizar un tratamiento con...
Name: text, dtype: object

In [16]:
raw_exam_df.tail(10)

2269              alarga el intervalo interdosis a 20 h).
2270    4. 40 mg/kg/20 h (solo se alarga el intervalo ...
2271                                         sis a 20 h).
2272    185. En un paciente de quién se conoce que ten...
2273    unos años un Mantoux negativo se produce un vi...
2274                      del mismo. La primera reacción:
2275                         1. Es un verdadero negativo.
2276                             2. Es un falso negativo.
2277                         3. Es un verdadero positivo.
2278                             4. Es un falso positivo.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [17]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [18]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. Pregunta asociada a la imagen 1. Paciente m...
1    1. Tiene una mejoría de los síntomas al desper...
2    2. Refiere una pérdida de visión, como una cor...
3    3. Sería necesario realizar un tratamiento con...
4    4. Debe realizarse tratamiento quirúrgico medi...
5    2. Pregunta asociada a la imagen 2. Profesora ...
6                                  1. Pólipos vocales.
7                     2. Carcinoma de cuerdas vocales.
8                                  3. Nódulos vocales.
9                                  4. Quistes vocales.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [19]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 20
Number of rows not ending with '.', ':' or '?' = 20
[80, 87, 89, 91, 93, 139, 371, 502, 558, 609, 612, 619, 657, 663, 678, 680, 686, 748, 896, 934]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [20]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [21]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [22]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    # rows_fix: list[int] = []
    # exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [23]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 925
925


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [24]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [25]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [26]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [27]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [28]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. Pregunta asociada a la imagen 1. Paciente m...,1. Tiene una mejoría de los síntomas al desper...,"2. Refiere una pérdida de visión, como una cor...",3. Sería necesario realizar un tratamiento con...,4. Debe realizarse tratamiento quirúrgico medi...
1,2. Pregunta asociada a la imagen 2. Profesora ...,1. Pólipos vocales.,2. Carcinoma de cuerdas vocales.,3. Nódulos vocales.,4. Quistes vocales.
2,3. Pregunta asociada a la imagen 3. Mujer de 6...,1. Leiomiosarcoma.,2. Tumor del estroma gastrointestinal.,3. Schwannoma.,4. Adenocarcinoma poco diferenciado.
3,4. Pregunta asociada a la imagen 4. Hombre de ...,1. Pioderma gangrenoso.,2. Pénfigo vulgar.,3. Infección micótica.,4. Infección por el virus del herpes simple.
4,5. Pregunta asociada a la imagen 5. Las célula...,1. Mieloma secretor de IgA leucemizado.,2. Linfoma folicular leucemizado.,3. Leucemia aguda mieloide.,4. Leucemia aguda linfoide.
5,6. Pregunta asociada a la imagen 6. Hombre de ...,1. Cirrosis.,2. Infiltración por carcinoma.,3. Amiloidosis.,4. Hepatitis alcohólica.
6,7. Pregunta asociada a la imagen 7. Varón de 5...,1. Metástasis mediastínica de carcinoma sarcom...,2. Sarcoma de nervio periférico.,3. Schwannoma.,4. Timoma.
7,8. Pregunta asociada a la imagen 8. Mujer de 4...,1. Análisis inmunológico: Ac. anti-cN1A positi...,2. Análisis inmunológico: Ac. anti-TIF-1 gamma...,3. Análisis inmunológico: Ac. anti-mitocondria...,4. Análisis inmunológico: Ac. anti-HMG-CoA red...
8,9. Pregunta asociada a la imagen 9. Mujer de 8...,1. Queratoacantoma.,2. Carcinoma basocelular.,3. Queratosis actínica hipertrófica.,4. Herpes simple crónico.
9,10. Pregunta asociada a la imagen 10. Varón de...,1. Prednisona oral.,2. Leflunomida.,3. Adalimumab.,4. Rituximab.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [29]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [30]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [31]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [32]:
answers_df_clean.head(10)

,V,RC
0,1,1.0
1,2,3.0
2,3,2.0
3,4,4.0
4,5,3.0
5,6,3.0
6,7,4.0
7,8,4.0
8,9,2.0
9,10,3.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [33]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. Pregunta asociada a la imagen 1. Paciente m...,1. Tiene una mejoría de los síntomas al desper...,"2. Refiere una pérdida de visión, como una cor...",3. Sería necesario realizar un tratamiento con...,4. Debe realizarse tratamiento quirúrgico medi...,1.0
1,2. Pregunta asociada a la imagen 2. Profesora ...,1. Pólipos vocales.,2. Carcinoma de cuerdas vocales.,3. Nódulos vocales.,4. Quistes vocales.,3.0
2,3. Pregunta asociada a la imagen 3. Mujer de 6...,1. Leiomiosarcoma.,2. Tumor del estroma gastrointestinal.,3. Schwannoma.,4. Adenocarcinoma poco diferenciado.,2.0
3,4. Pregunta asociada a la imagen 4. Hombre de ...,1. Pioderma gangrenoso.,2. Pénfigo vulgar.,3. Infección micótica.,4. Infección por el virus del herpes simple.,4.0
4,5. Pregunta asociada a la imagen 5. Las célula...,1. Mieloma secretor de IgA leucemizado.,2. Linfoma folicular leucemizado.,3. Leucemia aguda mieloide.,4. Leucemia aguda linfoide.,3.0
5,6. Pregunta asociada a la imagen 6. Hombre de ...,1. Cirrosis.,2. Infiltración por carcinoma.,3. Amiloidosis.,4. Hepatitis alcohólica.,3.0
6,7. Pregunta asociada a la imagen 7. Varón de 5...,1. Metástasis mediastínica de carcinoma sarcom...,2. Sarcoma de nervio periférico.,3. Schwannoma.,4. Timoma.,4.0
7,8. Pregunta asociada a la imagen 8. Mujer de 4...,1. Análisis inmunológico: Ac. anti-cN1A positi...,2. Análisis inmunológico: Ac. anti-TIF-1 gamma...,3. Análisis inmunológico: Ac. anti-mitocondria...,4. Análisis inmunológico: Ac. anti-HMG-CoA red...,4.0
8,9. Pregunta asociada a la imagen 9. Mujer de 8...,1. Queratoacantoma.,2. Carcinoma basocelular.,3. Queratosis actínica hipertrófica.,4. Herpes simple crónico.,2.0
9,10. Pregunta asociada a la imagen 10. Varón de...,1. Prednisona oral.,2. Leflunomida.,3. Adalimumab.,4. Rituximab.,3.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [34]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [35]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [36]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [37]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. Pregunta asociada a la imagen 1. Paciente m...,1. Tiene una mejoría de los síntomas al desper...,"2. Refiere una pérdida de visión, como una cor...",3. Sería necesario realizar un tratamiento con...,4. Debe realizarse tratamiento quirúrgico medi...,1,2020
1,2. Pregunta asociada a la imagen 2. Profesora ...,1. Pólipos vocales.,2. Carcinoma de cuerdas vocales.,3. Nódulos vocales.,4. Quistes vocales.,3,2020
2,3. Pregunta asociada a la imagen 3. Mujer de 6...,1. Leiomiosarcoma.,2. Tumor del estroma gastrointestinal.,3. Schwannoma.,4. Adenocarcinoma poco diferenciado.,2,2020
3,4. Pregunta asociada a la imagen 4. Hombre de ...,1. Pioderma gangrenoso.,2. Pénfigo vulgar.,3. Infección micótica.,4. Infección por el virus del herpes simple.,4,2020
4,5. Pregunta asociada a la imagen 5. Las célula...,1. Mieloma secretor de IgA leucemizado.,2. Linfoma folicular leucemizado.,3. Leucemia aguda mieloide.,4. Leucemia aguda linfoide.,3,2020
5,6. Pregunta asociada a la imagen 6. Hombre de ...,1. Cirrosis.,2. Infiltración por carcinoma.,3. Amiloidosis.,4. Hepatitis alcohólica.,3,2020
6,7. Pregunta asociada a la imagen 7. Varón de 5...,1. Metástasis mediastínica de carcinoma sarcom...,2. Sarcoma de nervio periférico.,3. Schwannoma.,4. Timoma.,4,2020
7,8. Pregunta asociada a la imagen 8. Mujer de 4...,1. Análisis inmunológico: Ac. anti-cN1A positi...,2. Análisis inmunológico: Ac. anti-TIF-1 gamma...,3. Análisis inmunológico: Ac. anti-mitocondria...,4. Análisis inmunológico: Ac. anti-HMG-CoA red...,4,2020
8,9. Pregunta asociada a la imagen 9. Mujer de 8...,1. Queratoacantoma.,2. Carcinoma basocelular.,3. Queratosis actínica hipertrófica.,4. Herpes simple crónico.,2,2020
9,10. Pregunta asociada a la imagen 10. Varón de...,1. Prednisona oral.,2. Leflunomida.,3. Adalimumab.,4. Rituximab.,3,2020


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [38]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 12
Number of rows in filtered df = 12


**2. Showing the filtered DataFrame**

Using head and tail

In [39]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,26. La adaptación metabólica al inicio del ayu...,1. Los aminoácidos ramificados son degradados ...,2. La alanina liberada es captada por el hígad...,3. La glutamina es transformada en los enteroc...,4. Los aminoácidos son utilizados en gran part...,2,2020
1,"27. El 2,3-BPG (2,3-bisfosfoglicerato) es un c...","1. La concentración de 2,3-BPG en los eritroci...","2. La presencia de 2,3-BPG apenas influye en l...","3. La adición de 2,3-BPG a la sangre almacenad...",4. La hemoglobina fetal tiene mayor afinidad p...,4,2020
2,28. En cuanto a las interrelaciones entre la p...,1. La resistencia es directamente proporcional...,"2. En el flujo laminar, la velocidad del flujo...",3. Cambios pequeños en el diámetro de un vaso ...,4. Si se añaden vasos sanguíneos en paralelo e...,4,2020
3,29. En relación con la autorregulación de la f...,1. Los valores de FG y FSR se mantienen relati...,"2. Se autorregulan en paralelo, pero en cierta...",3. La reducción del cloruro de sodio en la mác...,4. La angiotensina II ejerce una acción vasoco...,2,2020
4,31. El resultado de la gasometría arterial de...,"1. Ninguna, el pH es normal.",2. Acidosis respiratoria.,3. Acidosis respiratoria y alcalosis metabólica.,4. Alcalosis metabólica.,3,2020
5,32. ¿Cuál de las siguientes consecuencias pued...,1. Al ser una mutación intrónica no tiene ning...,2. Al ser una transición (los dos nucleótidos ...,3. Aunque es una mutación intrónica produce la...,4. La mutación en este nucleótido puede afecta...,4,2020
6,33. Cuando un mismo cuadro clínico (o con pequ...,1. Heterogeneidad de locus.,2. Heterogeneidad alélica.,3. Heterogeneidad genética.,4. Impronta parental.,2,2020
7,35. Los receptores para los fragmentos Fc de l...,1. La IgG se une al receptor III para Fc-gamma...,2. La IgA se une al receptor poli-Ig (pIgR) de...,3. La IgM se une al receptor Fc neonatal (FcRn...,4. La IgE se une al receptor para Fc-epsilon (...,3,2020
8,37. En relación con la inmunoterapia basada en...,1. Está aprobada para el tratamiento de leucem...,2. Un efecto secundario grave de esta terapia ...,3. El receptor CAR reconoce CD19 de forma rest...,4. Se asocia a hipergammaglobulinemia permanen...,2,2020
9,78. Referida a la parotiditis epidémica en la ...,1. El virus de la parotiditis solo presenta un...,2. La vacuna de la parotiditis está incluida e...,3. La orquitis como complicación es más frecue...,4. La pancreatitis puede producirse en la paro...,3,2020


In [40]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
2,28. En cuanto a las interrelaciones entre la p...,1. La resistencia es directamente proporcional...,"2. En el flujo laminar, la velocidad del flujo...",3. Cambios pequeños en el diámetro de un vaso ...,4. Si se añaden vasos sanguíneos en paralelo e...,4,2020
3,29. En relación con la autorregulación de la f...,1. Los valores de FG y FSR se mantienen relati...,"2. Se autorregulan en paralelo, pero en cierta...",3. La reducción del cloruro de sodio en la mác...,4. La angiotensina II ejerce una acción vasoco...,2,2020
4,31. El resultado de la gasometría arterial de...,"1. Ninguna, el pH es normal.",2. Acidosis respiratoria.,3. Acidosis respiratoria y alcalosis metabólica.,4. Alcalosis metabólica.,3,2020
5,32. ¿Cuál de las siguientes consecuencias pued...,1. Al ser una mutación intrónica no tiene ning...,2. Al ser una transición (los dos nucleótidos ...,3. Aunque es una mutación intrónica produce la...,4. La mutación en este nucleótido puede afecta...,4,2020
6,33. Cuando un mismo cuadro clínico (o con pequ...,1. Heterogeneidad de locus.,2. Heterogeneidad alélica.,3. Heterogeneidad genética.,4. Impronta parental.,2,2020
7,35. Los receptores para los fragmentos Fc de l...,1. La IgG se une al receptor III para Fc-gamma...,2. La IgA se une al receptor poli-Ig (pIgR) de...,3. La IgM se une al receptor Fc neonatal (FcRn...,4. La IgE se une al receptor para Fc-epsilon (...,3,2020
8,37. En relación con la inmunoterapia basada en...,1. Está aprobada para el tratamiento de leucem...,2. Un efecto secundario grave de esta terapia ...,3. El receptor CAR reconoce CD19 de forma rest...,4. Se asocia a hipergammaglobulinemia permanen...,2,2020
9,78. Referida a la parotiditis epidémica en la ...,1. El virus de la parotiditis solo presenta un...,2. La vacuna de la parotiditis está incluida e...,3. La orquitis como complicación es más frecue...,4. La pancreatitis puede producirse en la paro...,3,2020
10,81. En relación con la anemia de Fanconi indiq...,1. La anemia aplásica suele estar ya presente ...,2. Son típicas las malformaciones esqueléticas.,3. El trasplante de progenitores hematopoyétic...,4. Los andrógenos suelen producir una respuest...,1,2020
11,169. ¿Cuál de los siguientes iones urinarios f...,1. Citrato.,2. Magnesio.,3. Sodio.,4. Sulfato.,3,2020


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [41]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [42]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [43]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT id FROM questions WHERE question = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [44]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], n+1, option, n+1 == question[6]))
            bir_warehouse.commit()